Markuis Graylee adalah seorang metuber yang membuat konten mengenai produk-produk elektronik. Untuk salah satu ide kontennya, ia ingin mereview smartphone yang telah dirilis oleh beberapa perusahaan. Sebagai pendukung penelitian kontennya, ia memiliki akses pada sebuah dataset yang berisi informasi-informasi relevan yang dapat diteliti untuk menilai produk-produk yang dirilis perusahaan. Markuis juga ingin menemukan pola-pola atau hubungan yang dimiliki oleh produk-produk tersebut agar ia dapat mengetahui tren yang ada pada dunia per-smartphone-an.

Atribut: 
- battery_power
- clock_speed
- ram
- n_cores
- use_time
- px_width
- px_height
- brand
- 5g
- grade
- price

Gunakan alpha = 0.05

No. 6 Hipotesis 2 Sampel

Markuis berasumsi setengah bagian pertama dataset adalah smartphone generasi sebelumnya dan setengah bagian terakhir adalah smartphone generasi sekarang.

- Periksa apakah rata-rata jumlah core (n_cores) smartphone generasi sebelumnya sama dengan jumlah core smartphone generasi sekarang?
- Bagaimana dengan harga smartphone, apakah harga smartphone generasi sekarang lebih mahal 100 dari generasi sebelumnya?
- Apakah variansi dari tinggi smartphone (px_height) sama pada kedua generasi?
- Apakah proporsi kapasitas baterai (battery_power) smartphone yang lebih dari 2030 pada smartphone generasi sebelumnya lebih besar daripada proporsi kapasitas baterai (battery_power) smartphone yang lebih dari 2030 pada smartphone generasi sekarang?

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.weightstats import ttest_ind
%run util.py

In [ ]:
# Mean
def myMean2(data, atribut):
    # mean = Σx / n
    n = data[atribut].shape[0]
    jumlah = 0
    for i in range(n):
        jumlah += data[atribut][i]
    return jumlah/n

# Variance
def myVariance2(data, atribut):
    # variance = Σ(x - mean)^2 / (n - 1)
    n = data[atribut].shape[0]
    mean = myMean2(data, atribut)
    jumlah = 0
    for i in range(n):
        jumlah += (data[atribut][i] - mean) ** 2
    return jumlah / (n - 1)

In [ ]:
# Mengganti outliers dengan nilai median
def changeOutliers(data, atribut):
    Q1 = myQ1(data, atribut)
    Q3 = myQ3(data, atribut)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    data.loc[(data[atribut] < lower_bound) | (data[atribut] > upper_bound), atribut] = data[atribut].median()
    return data

# Mengganti outliers dengan nilai median
DATA = changeOutliers(DATA, "battery_power")
DATA = changeOutliers(DATA, "clock_speed")
DATA = changeOutliers(DATA, "ram")
DATA = changeOutliers(DATA, "n_cores")
DATA = changeOutliers(DATA, "use_time")
DATA = changeOutliers(DATA, "px_width")
DATA = changeOutliers(DATA, "px_height")
DATA = changeOutliers(DATA, "5g")
DATA = changeOutliers(DATA, "price")

In [ ]:
# Pisah data smartphone generasi sebelumnya dan generasi sekarang
n: int = DATA.shape[0]
old_smartphone: pd.DataFrame = DATA.loc[:(n / 2 - 1)]
new_smartphone: pd.DataFrame = DATA.loc[(n / 2):]
new_smartphone.reset_index(drop=True, inplace=True)


# 6.1 Periksa apakah rata-rata jumlah core (n_cores) smartphone generasi sebelumnya sama dengan jumlah core smartphone generasi sekarang?  

Sebelum melakukan tes hipotesis pada rata-rata, kita perlu mengecek apakah variansi populasi dari kedua generasi sama atau tidak karena tidak diketahui informasi mengenai variansi populasi.

Langkah-langkah:  
1. Hipotesis nol:  
   $$H_0 : σ^2_1 = σ^2_2$$  
   $σ^2_1$: variansi populasi dari jumlah core smartphone generasi sekarang  
   $σ^2_2$: variansi populasi dari jumlah core smartphone generasi sebelumnya  

2. Hipotesis alternatif:  
   $$H_1 : σ^2_1 ≠ σ^2_2$$  

3. Tingkat signifikan:
   $$α = 0.05$$  

In [ ]:
# Tingkat signifikan
alpha = 0.05

4. Uji statistik:  
   Pada kasus ini, digunakan uji hipotesis variansi pada 2 sampel.  
   Statistik nilai-f untuk variansi 2 sampel:  
   $$f = \frac{s^2_1}{s^2_2}$$  
   $s^2_1$: variansi sampel dari jumlah core smartphone generasi sekarang  
   $s^2_2$: variansi sampel dari jumlah core smartphone generasi sebelumnya  

   Daerah kritis:  
   $$f_{critical} < f_{1-α/2}(v_1, v_2) \: atau \: f_{critical} > f_{α/2}(v_1, v_2)$$
   untuk:
   $$α = 0.05$$  
   $$v_1 = n_1 - 1 = 1000 - 1 = 999$$  
   $$v_2 = n_2 - 1= 1000 - 1 = 999$$  
   $n_1$: banyak sampel dari jumlah core smartphone generasi sekarang  
   $n_2$: banyak sampel dari jumlah core smartphone generasi sebelumnya 

In [ ]:
# batas daerah kritis
print("batas kiri:", stats.f.isf(1 - alpha / 2, 999, 999).round(3))
print("batas kanan:", stats.f.isf(alpha / 2, 999, 999).round(3))

Berdasarkan nilai batas yang diperoleh, diperoleh daerah kritis:  
$$f_{critical} < 0.883 \: atau \: f_{critical} > 1.132$$  

5. Nilai uji statistik dan p-value

In [ ]:
# Variansi sampel
my_var_1 = myVariance2(new_smartphone, "n_cores")
my_var_2 = myVariance2(old_smartphone, "n_cores")
library_s1 = new_smartphone["n_cores"].var()
library_s2 = old_smartphone["n_cores"].var()

# nilai-f dan p-value
my_f = my_var_1 / my_var_2
my_p_value = 2 * min(stats.f.sf(my_f, 999, 999), 1 - stats.f.sf(my_f, 999, 999))
library_f = library_s1 / library_s2
library_p_value = 2 * min(stats.f.sf(library_f, 999, 999), 1 - stats.f.sf(library_f, 999, 999))
print("my_f:", my_f.round(3))
print("my_p_value:", my_p_value.round(3))
print("library_f:", library_f.round(3))
print("library_p_value:", library_p_value.round(3))

6. Berdasarkan perhitungan, diperoleh:  
   f = 0.984  
   p-value = 0.803
   
   f berada di luar critical region  
   p-value > α  
   Oleh karena itu, $H_0$ tidak ditolak. Variansi dari jumlah core smartphone (n_cores) sama pada kedua generasi.  

Karena kesamaan variansi sudah diketahui, kita dapat melakukan tes hipotesis untuk rata-rata

Langkah-langkah:  
1. Hipotesis nol:  
   $$H_0 : μ_1 = μ_2$$  
   $μ_1$: rata-rata populasi dari jumlah core smartphone generasi sekarang  
   $μ_2$: rata-rata populasi dari jumlah core smartphone generasi sebelumnya  

2. Hipotesis alternatif:  
   $$H_1 : μ_1 ≠ μ_2$$  

3. Tingkat signifikan:
   $$α = 0.05$$  

In [ ]:
# Tingkat signifikan
alpha = 0.05

4. Uji statistik:  
   Pada kasus ini, digunakan uji hipotesis rata-rata pada 2 sampel.  
   Statistik nilai-t untuk variansi 2 sampel:  
   $$t = \frac{(\bar{x}_1 - \bar{x}_2) - d_0}{s_p\sqrt{\frac{1}{n_1} + \frac{1}{n_2}}}$$  
   $$v = n_1 + n_2 - 2$$
   $$s_p = \sqrt{\frac{(n_1 - 1) s^2_1 + (n_2 - 1) s^2_2}{n_1 + n_2 - 2}}$$
   $s^2_1$: variansi sampel dari jumlah core smartphone generasi sekarang  
   $s^2_2$: variansi sampel dari jumlah core smartphone generasi sebelumnya  

   Daerah kritis:  
   $$t_{critical} < -t_{α/2} \: atau \: t_{critical} > t_{α/2}$$
   untuk:
   $$α = 0.05$$  

In [ ]:
# batas daerah kritis
print("batas kiri:", -stats.t.isf(alpha / 2, 1998).round(3))
print("batas kanan:", stats.t.isf(alpha / 2, 1998).round(3))

Berdasarkan nilai batas yang diperoleh, diperoleh daerah kritis:  
$$t_{critical} < -1.961 \: atau \: t_{critical} > 1.961$$  

5. Nilai uji statistik dan p-value

In [ ]:
# Rata-rata sampel
my_xbar1 = myMean2(new_smartphone, "n_cores")
my_xbar2 = myMean2(old_smartphone, "n_cores")
library_xbar1 = new_smartphone["n_cores"].mean()
library_xbar2 = old_smartphone["n_cores"].mean()

# Variansi sampel
my_var_1 = myVariance2(new_smartphone, "n_cores")
my_var_2 = myVariance2(old_smartphone, "n_cores")
library_s1 = new_smartphone["n_cores"].var()
library_s2 = old_smartphone["n_cores"].var()

# d0, n1, n2, v, sp
d0 = 0
n1 = new_smartphone["n_cores"].shape[0]
n2 = old_smartphone["n_cores"].shape[0]
v = n1 + n2 - 2
sp = np.sqrt(((n1 - 1) * my_var_1 + (n2 - 1) * my_var_2) / v)

# nilai-t dan p-value
my_t = (my_xbar1 - my_xbar2 - d0) / (sp * np.sqrt(1 / n1 + 1 / n2))
my_p_value = 2 * stats.t.sf(np.abs(my_t), v)
library_t, library_p_value = stats.ttest_ind(new_smartphone["n_cores"], old_smartphone["n_cores"], equal_var=True, alternative="two-sided")
print("my_t:", my_t.round(3))
print("my_p_value:", my_p_value.round(3))
print("library_t:", library_t.round(3))
print("library_p_value:", library_p_value.round(3))

6. Berdasarkan perhitungan, diperoleh:  
   t = -1.038  
   p-value = 0.299  
   
   t berada di luar critical region  
   p-value > α  
   Oleh karena itu, $H_0$ tidak ditolak. Rata-rata dari jumlah core (n_cores) sama pada kedua generasi.  

# 6.2 Bagaimana dengan harga smartphone, apakah harga smartphone generasi sekarang lebih mahal 100 dari generasi sebelumnya?

Sebelum melakukan tes hipotesis pada rata-rata, kita perlu mengecek apakah variansi populasi dari kedua generasi sama atau tidak karena tidak diketahui informasi mengenai variansi populasi.

Langkah-langkah:  
1. Hipotesis nol:  
   $$H_0 : σ^2_1 = σ^2_2$$  
   $σ^2_1$: variansi populasi dari harga smartphone generasi sekarang  
   $σ^2_2$: variansi populasi dari harga smartphone generasi sebelumnya  

2. Hipotesis alternatif:  
   $$H_1 : σ^2_1 ≠ σ^2_2$$  

3. Tingkat signifikan:
   $$α = 0.05$$  

In [ ]:
# Tingkat signifikan
alpha = 0.05

4. Uji statistik:  
   Pada kasus ini, digunakan uji hipotesis variansi pada 2 sampel.  
   Statistik nilai-f untuk variansi 2 sampel:  
   $$f = \frac{s^2_1}{s^2_2}$$  
   $s^2_1$: variansi sampel dari harga smartphone generasi sekarang  
   $s^2_2$: variansi sampel dari harga smartphone generasi sebelumnya  

   Daerah kritis:  
   $$f_{critical} < f_{1-α/2}(v_1, v_2) \: atau \: f_{critical} > f_{α/2}(v_1, v_2)$$
   untuk:
   $$α = 0.05$$  
   $$v_1 = n_1 - 1 = 1000 - 1 = 999$$  
   $$v_2 = n_2 - 1= 1000 - 1 = 999$$  
   $n_1$: banyak sampel dari harga smartphone generasi sekarang  
   $n_2$: banyak sampel dari harga smartphone generasi sebelumnya 

In [ ]:
# batas daerah kritis
print("batas kiri:", stats.f.isf(1 - alpha / 2, 999, 999).round(3))
print("batas kanan:", stats.f.isf(alpha / 2, 999, 999).round(3))

Berdasarkan nilai batas yang diperoleh, diperoleh daerah kritis:  
$$f_{critical} < 0.883 \: atau \: f_{critical} > 1.132$$  

5. Nilai uji statistik dan p-value

In [ ]:
# Variansi sampel
my_var_1 = myVariance2(new_smartphone, "price")
my_var_2 = myVariance2(old_smartphone, "price")
library_s1 = new_smartphone["price"].var()
library_s2 = old_smartphone["price"].var()

# nilai-f dan p-value
my_f = my_var_1 / my_var_2
my_p_value = 2 * min(stats.f.sf(my_f, 999, 999), 1 - stats.f.sf(my_f, 999, 999))
library_f = library_s1 / library_s2
library_p_value = 2 * min(stats.f.sf(library_f, 999, 999), 1 - stats.f.sf(library_f, 999, 999))
print("my_f:", my_f.round(3))
print("my_p_value:", my_p_value.round(3))
print("library_f:", library_f.round(3))
print("library_p_value:", library_p_value.round(3))

6. Berdasarkan perhitungan, diperoleh:  
   f = 1.018  
   p-value = 0.775
   
   f berada di luar critical region  
   p-value > α  
   Oleh karena itu, $H_0$ tidak ditolak. Variansi dari jumlah core smartphone (n_cores) sama pada kedua generasi.  

Karena kesamaan variansi sudah diketahui, kita dapat melakukan tes hipotesis untuk rata-rata

Langkah-langkah:  
1. Hipotesis nol:  
   $$H_0 : μ_1 - μ_2 = 100$$  
   $μ_1$: rata-rata populasi dari harga smartphone generasi sekarang  
   $μ_2$: rata-rata populasi dari harga smartphone generasi sebelumnya  

2. Hipotesis alternatif:  
   $$H_1 : μ_1 - μ_2 ≠ 100$$  

3. Tingkat signifikan:
   $$α = 0.05$$  

In [ ]:
# Tingkat signifikan
alpha = 0.05

4. Uji statistik:  
   Pada kasus ini, digunakan uji hipotesis rata-rata pada 2 sampel.  
   Statistik nilai-t untuk variansi 2 sampel:  
   $$t = \frac{(\bar{x}_1 - \bar{x}_2) - d_0}{s_p\sqrt{\frac{1}{n_1} + \frac{1}{n_2}}}$$  
   $$v = n_1 + n_2 - 2$$
   $$s_p = \sqrt{\frac{(n_1 - 1) s^2_1 + (n_2 - 1) s^2_2}{n_1 + n_2 - 2}}$$
   $s^2_1$: variansi sampel dari harga smartphone generasi sekarang  
   $s^2_2$: variansi sampel dari harga smartphone generasi sebelumnya  

   Daerah kritis:  
   $$t_{critical} < -t_{α/2} \: atau \: t_{critical} > t_{α/2}$$
   untuk:
   $$α = 0.05$$  

In [ ]:
# batas daerah kritis
print("batas kiri:", -stats.t.isf(alpha / 2, 1998).round(3))
print("batas kanan:", stats.t.isf(alpha / 2, 1998).round(3))

Berdasarkan nilai batas yang diperoleh, diperoleh daerah kritis:  
$$t_{critical} < -1.961 \: atau \: t_{critical} > 1.961$$  

5. Nilai uji statistik dan p-value

In [ ]:
# Rata-rata sampel
my_xbar1 = myMean2(new_smartphone, "price")
my_xbar2 = myMean2(old_smartphone, "price")
library_xbar1 = new_smartphone["price"].mean()
library_xbar2 = old_smartphone["price"].mean()
print("my_xbar1:", my_xbar1.round(3))
print("my_xbar2:", my_xbar2.round(3))
print("library_xbar1:", library_xbar1.round(3))
print("library_xbar2:", library_xbar2.round(3))

# Variansi sampel
my_var_1 = myVariance2(new_smartphone, "price")
my_var_2 = myVariance2(old_smartphone, "price")
library_s1 = new_smartphone["price"].var()
library_s2 = old_smartphone["price"].var()

# d0, n1, n2, v, sp
d0 = 100
n1 = new_smartphone["price"].shape[0]
n2 = old_smartphone["price"].shape[0]
v = n1 + n2 - 2
sp = np.sqrt(((n1 - 1) * my_var_1 + (n2 - 1) * my_var_2) / v)

# nilai-t dan p-value
my_t = (my_xbar1 - my_xbar2 - d0) / (sp * np.sqrt(1 / n1 + 1 / n2))
my_p_value = 2 * stats.t.sf(np.abs(my_t), v)
library_t, library_p_value = stats.ttest_ind(new_smartphone["price"] - d0, old_smartphone["price"], equal_var=True, alternative="two-sided")
print("my_t:", my_t.round(3))
print("my_p_value:", my_p_value.round(3))
print("library_t:", library_t.round(3))
print("library_p_value:", library_p_value.round(3))

6. Berdasarkan perhitungan, diperoleh:  
   t = -1.196  
   p-value = 0.232  
   
   t berada di luar critical region  
   p-value > α  
   Oleh karena itu, $H_0$ tidak ditolak. Rata-rata harga (price) smartphone generasi sekarang lebih mahal 100 dari generasi sebelumnya.  

# 6.3 Apakah variansi dari tinggi smartphone (px_height) sama pada kedua generasi?

Langkah-langkah:  
1. Hipotesis nol:  
   $$H_0 : σ^2_1 = σ^2_2$$  
   $σ^2_1$: variansi populasi dari tinggi smartphone generasi sekarang  
   $σ^2_2$: variansi populasi dari tinggi smartphone generasi sebelumnya  

2. Hipotesis alternatif:  
   $$H_1 : σ^2_1 ≠ σ^2_2$$  

3. Tingkat signifikan:
   $$α = 0.05$$  

In [ ]:
# Tingkat signifikan
alpha = 0.05

4. Uji statistik:  
   Pada kasus ini, digunakan uji hipotesis variansi pada 2 sampel.  
   Statistik nilai-f untuk variansi 2 sampel:  
   $$f = \frac{s^2_1}{s^2_2}$$  
   $s^2_1$: variansi sampel dari tinggi smartphone generasi sekarang  
   $s^2_2$: variansi sampel dari tinggi smartphone generasi sebelumnya  

   Daerah kritis:  
   $$f_{critical} < f_{1-α/2}(v_1, v_2) \: atau \: f_{critical} > f_{α/2}(v_1, v_2)$$
   untuk:
   $$α = 0.05$$  
   $$v_1 = n_1 - 1 = 1000 - 1 = 999$$  
   $$v_2 = n_2 - 1= 1000 - 1 = 999$$  
   $n_1$: banyak sampel dari tinggi smartphone generasi sekarang  
   $n_2$: banyak sampel dari tinggi smartphone generasi sebelumnya 

In [ ]:
# batas daerah kritis
print("batas kiri:", stats.f.isf(1 - alpha / 2, 999, 999).round(3))
print("batas kanan:", stats.f.isf(alpha / 2, 999, 999).round(3))

Berdasarkan nilai batas yang diperoleh, diperoleh daerah kritis:  
$$f_{critical} < 0.883 \: atau \: f_{critical} > 1.132$$  

5. Nilai uji statistik dan p-value

In [ ]:
# Variansi sampel
my_var_1 = myVariance2(new_smartphone, "px_height")
my_var_2 = myVariance2(old_smartphone, "px_height")
library_s1 = new_smartphone["px_height"].var()
library_s2 = old_smartphone["px_height"].var()

# nilai-f dan p-value
my_f = my_var_1 / my_var_2
my_p_value = 2 * min(stats.f.sf(my_f, 999, 999), 1 - stats.f.sf(my_f, 999, 999))
library_f = library_s1 / library_s2
library_p_value = 2 * min(stats.f.sf(library_f, 999, 999), 1 - stats.f.sf(library_f, 999, 999))
print("my_f:", my_f.round(3))
print("my_p_value:", my_p_value.round(3))
print("library_f:", library_f.round(3))
print("library_p_value:", library_p_value.round(3))

6. Berdasarkan perhitungan, diperoleh:  
   f = 0.951  
   p-value = 0.428
   
   f berada di luar critical region  
   p-value > α  
   Oleh karena itu, $H_0$ tidak ditolak. Variansi dari tinggi smartphone (px_height) sama pada kedua generasi.  

# 6.4 Apakah proporsi kapasitas baterai (battery_power) smartphone yang lebih dari 2030 pada smartphone generasi sebelumnya lebih besar daripada proporsi kapasitas baterai (battery_power) smartphone yang lebih dari 2030 pada smartphone generasi sekarang?

Langkah-langkah:  
1. Hipotesis nol:  
   $$H_0 : p_1 = p_2$$  
   $p_1$: proporsi populasi dari kapasitas baterai smartphone yang lebih dari 2030 pada smartphone generasi sekarang  
   $p_2$: proporsi populasi dari kapasitas baterai smartphone yang lebih dari 2030 pada smartphone generasi sebelumnya  

2. Hipotesis alternatif:  
   $$H_1 : p_1 < p_2$$  

3. Tingkat signifikan:
   $$α = 0.05$$  

In [ ]:
# Tingkat signifikan
alpha = 0.05

4. Uji statistik:  
   Pada kasus ini, digunakan uji hipotesis proporsi pada 2 sampel.  
   Statistik nilai-z untuk variansi 2 sampel:  
   $$z = \frac{\hat{p_1} - \hat{p_2}}{\sqrt{\hat{p}\hat{q}(\frac{1}{n_1} + \frac{1}{n_2})}}$$  
   $$\hat{p} = \frac{x_1 + x_2}{n_1 + n_2}$$
   $$\hat{q} = 1 - \hat{p}$$
   $\hat{p_1}$: proporsi sampel dari kapasitas baterai smartphone yang lebih dari 2030 pada smartphone generasi sekarang  
   $\hat{p_2}$: proporsi sampel dari kapasitas baterai smartphone yang lebih dari 2030 pada smartphone generasi sekarang  
   $x_1$: banyak sampel dari kapasitas baterai smartphone yang lebih dari 2030 pada smartphone generasi sekarang  
   $x_2$: banyak sampel dari kapasitas baterai smartphone yang lebih dari 2030 pada smartphone generasi sebelumnya  
   $n_1$: banyak sampel smartphone generasi sekarang  
   $n_2$: banyak sampel smartphone generasi sebelumnya  

   Daerah kritis:  
   $$z_{critical} < -z_α$$
   untuk:
   $$α = 0.05$$  

In [ ]:
# batas daerah kritis
print("batas kiri:", -stats.norm.isf(alpha).round(3))

Berdasarkan nilai batas yang diperoleh, diperoleh daerah kritis:  
$$z_{critical} < -1.645$$  

5. Nilai uji statistik dan P-value

In [ ]:
# Ambil semua data battery_power
new_battery_power: pd.Series = new_smartphone["battery_power"]
old_battery_power: pd.Series = old_smartphone["battery_power"]

# Ambil data battery_power yang lebih dari 2030
new_battery_power_2030: pd.Series = new_battery_power[new_battery_power > 2030]
old_battery_power_2030: pd.Series = old_battery_power[old_battery_power > 2030]

# n1, n2, p1, p2, p, q
x1 = new_battery_power_2030.shape[0]
x2 = old_battery_power_2030.shape[0]
n1 = new_battery_power.shape[0]
n2 = old_battery_power.shape[0]
p1 = x1 / n1
p2 = x2 / n2
p = (x1 + x2) / (n1 + n2)
q = 1 - p

# nilai-z dan p-value
my_z = (p1 - p2) / np.sqrt(p * q * (1 / n1 + 1 / n2))
my_p_value = stats.norm.cdf(my_z)
library_z, library_p_value = proportions_ztest([x1, x2], [n1, n2], alternative="smaller")
print("my_z:", my_z.round(3))
print("my_p_value:", my_p_value.round(3))
print("library_z:", library_z.round(3))
print("library_p_value:", library_p_value.round(3))


6. Berdasarkan perhitungan, diperoleh:  
   z = 0.614  
   p-value = 0.731
   
   z berada di luar critical region  
   p-value > α  
   Oleh karena itu, $H_0$ tidak ditolak. Proporsi kapasitas baterai (battery_power) smartphone yang lebih dari 2030 pada smartphone generasi sebelumnya sama dengan proporsi kapasitas baterai (battery_power) smartphone yang lebih dari 2030 pada smartphone generasi sekarang.  